In [1]:
import os
import csv

import numpy as np
from PIL import Image
import skimage
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
import random as rd

### Load data

In [2]:
image_dir_pos = 'train/images/pos'
image_dir_neg = 'train/images/neg'
csv_dir = 'train/labels_csv'
image_list_pos = os.listdir(image_dir_pos)
image_list_neg = os.listdir(image_dir_neg)
image_list = image_list_pos + image_list_neg
csv_list = os.listdir(csv_dir)

### Dictionary to store contour coordinates for each image

In [5]:
coordinates = {}
for csv_file in csv_list:
    if csv_file != '.DS_Store':
        image_name = csv_file.split('.')[0]
        with open(f"train/labels_csv/{csv_file}") as csvfile:
            reader = csv.reader(csvfile)
            row_image = []
            for row in reader:
                row_image.append(row)
        coordinates[image_name]=row_image

# Positiive image

### Cropping of positive image

In [6]:
for csv_file in csv_list:
    if csv_file != '.DS_Store':
        image_name = csv_file.split('.')[0]
        image = Image.open(f"train/images/pos/{image_name}.jpg")
        for i_ in range(len(coordinates[image_name])):
            i, j, h, l, d = coordinates[image_name][i_]
            i = int(i)
            j = int(j)
            h = int(h)
            l = int(l)
            left = j
            top = i
            right = j + l
            bottom = i + h
            im1 = image.crop((left, top, right, bottom))
            im1.save(f'/Users/lilianvalin/Desktop/image_crop_pos/{image_name}_{i_}.png', 'png')

KeyboardInterrupt: 

### Resize of positive image

In [8]:
image_dir_crop_pos = '/Users/lilianvalin/Desktop/image_crop_pos'
image_list_crop_pos = os.listdir(image_dir_crop_pos)

for image in image_list_crop_pos:
    if image != '.DS_Store':
        image_name = image.split('.')[0]
        image = Image.open(f"/Users/lilianvalin/Desktop/image_crop_pos/{image_name}.png")
        resize_image = image.resize((174, 301))
        resize_image.save(f'/Users/lilianvalin/Desktop/image_resize_pos/{image_name}.jpg')

### HOG calculation for cropped positive images

In [3]:
image_dir_resize_pos = '/Users/lilianvalin/Desktop/image_resize_pos'
image_list_resize_pos = os.listdir(image_dir_resize_pos)

hog_resize_pos = np.zeros((len(image_list_resize_pos), 53865))

for i_, image in enumerate(image_list_resize_pos) :
    if image != '.DS_Store':
        image_name = image.split('.')[0]
        image = Image.open(f"/Users/lilianvalin/Desktop/image_resize_pos/{image_name}.jpg")
        fd = hog(image, channel_axis=-1)
        hog_resize_pos[i_] = fd

In [4]:
print(hog_resize_pos.shape)

(971, 53865)


# Negative image

### Cropping of negative image

In [5]:
def create_train_neg (k):
    for csv_file in image_list_neg:
        if csv_file != '.DS_Store':
            image_name = csv_file.split('.')[0]
            image = Image.open(f"train/images/neg/{image_name}.jpg")
            width, height = image.size
            i = rd.randint(0, height)
            j = rd.randint(0, width)
            h = rd.randint(50, 300)
            l = rd.randint(50, 300)
            left = j
            top = i
            right = j + l
            bottom = i + h
            if left > 0 and top > 0 and right < width and bottom < height :
                im1 = image.crop((left, top, right, bottom))
                im1.save(f'image_crop_neg/{image_name}_{k}.jpg')

for k in range(20):
    create_train_neg(k)

### Resize of negative image

In [13]:
image_dir_crop_neg = '/Users/lilianvalin/Desktop/image_crop_neg'
image_list_crop_neg = os.listdir(image_dir_crop_neg)

for image in image_list_crop_neg:
    if image != '.DS_Store':
        image_name = image.split('.')[0]
        image = Image.open(f"/Users/lilianvalin/Desktop/image_crop_neg/{image_name}.png")
        resize_image = image.resize((174, 301))
        resize_image.save(f'/Users/lilianvalin/Desktop/image_resize_neg/{image_name}.jpg')

### HOG calculation for cropped negative images

In [5]:
image_dir_resize_neg = '/Users/lilianvalin/Desktop/image_resize_neg'
image_list_resize_neg = os.listdir(image_dir_resize_neg)

hog_resize_neg = np.zeros((len(image_list_resize_neg), 53865))

for i_, image in enumerate(image_list_resize_neg) :
    if image != '.DS_Store':
        image_name = image.split('.')[0]
        image = Image.open(f"/Users/lilianvalin/Desktop/image_resize_neg/{image_name}.jpg")
        fd = hog(image, channel_axis=-1)
        hog_resize_neg[i_] = fd

In [6]:
print(hog_resize_neg.shape)

(1823, 53865)


# List of HOG - X_train, y_train

In [7]:
# Normaliser les valeurs du HOG image positive
hog_resize_pos = (hog_resize_pos - hog_resize_pos.mean(axis=0)) / hog_resize_pos.std(axis=0)

# Écrire les HOGs dans un fichier CSV
with open('/Users/lilianvalin/Desktop/X_train.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(hog_resize_pos)

# Normaliser les valeurs du HOG image negative
hog_resize_neg = (hog_resize_neg - hog_resize_neg.mean(axis=0)) / hog_resize_neg.std(axis=0)

# Écrire les HOGs dans un fichier CSV
with open('/Users/lilianvalin/Desktop/X_train.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(hog_resize_neg)

In [ ]:
with open('/Users/lilianvalin/Desktop/X_train.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    X_train = np.array(list(reader)).astype(float)

In [ ]:
print(X_train.shape)

In [174]:
X_train = np.vstack((hog_resize_pos, hog_resize_neg))
y_train = np.zeros((len(X_train), 1) , dtype=int)
for y_ in range(len(hog_resize_pos)):
    y_train[y_] = 1

In [175]:
from sklearn.ensemble import RandomForestClassifier

# Initialisation d'un objet de la classe RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=200)

# Aplatir y_train
y_train = np.ravel(y_train)

# Entraînement du modèle avec les données d'entraînement
clf.fit(X_train, y_train)

# Utilisation du modèle entraîné pour faire des prédictions sur de nouvelles données
y_pred = clf.predict(X_train)

In [176]:
image = Image.open("/Users/lilianvalin/Desktop/image_resize_pos/abigotte_pos_001_0.jpg")
hog_one_image = np.zeros((1, 53865))
hog_one_image[0] = hog(image, channel_axis=-1)
print(clf.predict(hog_one_image))

[1]


In [167]:
def testing(image):
    hog_one_image = np.zeros((1, 53865))
    hog_one_image[0] = hog(image, channel_axis=-1)
    res = clf.predict(hog_one_image)
    return int(res)

In [177]:
print(testing(Image.open("/Users/lilianvalin/Desktop/image_resize_pos/abigotte_pos_001_0.jpg")))

1


In [130]:
def sliding_window(image = Image.open("test/000.jpg"), window_size = (174, 301), stride = 100):
    res = []
    # Obtient les dimensions de l'image
    width, height = image.size
    # Parcourt l'image avec la taille de fenêtre et le pas de parcours donnés
    for x in range(0, width - window_size[0] + 1, stride):
        for y in range(0, height - window_size[1] + 1, stride):
            # Extrait la région de l'image correspondant à la fenêtre
            region = image.crop((x, y, x + window_size[0], y + window_size[1]))
            # Fait quelque chose avec la région extraite, par exemple l'affiche
            resultat = testing(region)
            if resultat == 1 :
                region.show()
            res.append(resultat)
    return res

In [131]:
def resizing_window(nb_resize, image_name):
    image = Image.open(f"test/{image_name}.jpg")
    width, height = image.size
    for i_ in range(1, nb_resize):
        new_image = image.resize((width*i_, height*i_))
        print(sliding_window(new_image))

In [178]:
resizing_window(4, '001')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt


KeyboardInterrupt: 